In [1]:
import os
import pandas as pd
import cudf
import cupy as cp
from cuml.neighbors import NearestNeighbors
import numpy as np
from imblearn.under_sampling import OneSidedSelection

from common.env_config import config

# Giả sử X_train và y_train là dữ liệu của bạn

X_train = np.load("word2vec_train_vector.npy")
y_train = np.load("y_train.npy")
print(X_train.shape, y_train.shape)

# Chuyển đổi dữ liệu sang định dạng GPU
X_gpu = cudf.DataFrame(X_train)
y_gpu = cudf.Series(y_train)

# Sử dụng KNN trên GPU để tìm điểm lân cận nhanh hơn
print("Start training KNN ...")
knn_gpu = NearestNeighbors(n_neighbors=5)
knn_gpu.fit(X_gpu)
distances, indices = knn_gpu.kneighbors(X_gpu)

# Chuyển kết quả về CPU để sử dụng với OneSidedSelection
print("Exchange GPU to CPU")
distances_cpu = distances.values_host
indices_cpu = indices.values_host
X_cpu = X_gpu.to_pandas()
y_cpu = y_gpu.to_pandas()

(994372, 300) (994372,)
Start training KNN ...
Exchange GPU to CPU


In [ ]:
# Thực hiện OneSidedSelection trên CPU với kết quả từ GPU
print("Start training OneSidedSelection ...")
oss = OneSidedSelection(n_neighbors=1, n_seeds_S=200, n_jobs=-1)
X_resampled, y_resampled = oss.fit_resample(X_cpu, y_cpu)
print(X_resampled.shape, y_resampled.shape)

Start training OneSidedSelection ...
